In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64
from pymongo import MongoClient
from bson.json_util import dumps

from ProjectOneR import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "c0d3Us3r"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),    
    html.Hr(),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width=400, height=400)),
    html.Center(html.B(html.H1('Animal Rescue Dashboard', style={'color':'rgb(180,0,0)'}))),
    html.Hr(),
    #Create radio button options for sorting data by rescue type
    html.Div(html.Center(
        dcc.RadioItems( id='filter-type',           
            options=[ 
                {'label':'Water Rescue', 'value':'WR'},
                {'label':'Mountain/Wilderness Rescue','value':'MWR'},
                {'label':'Disaster Rescue or Individual Tracking','value':'DRIT'},
                {'label':'Reset/Clear','value':'RESET'},],
            value='RESET', 
            labelStyle={'display':'inline-block'},
                  style={'font-size': 14,'font-weight':'bold'})
   )),
    
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],        
        data=df.to_dict('records'),
        #Features to make the data table user-friendly
        editable=False,
        column_selectable=False,
        row_deletable=False,
        row_selectable=False,
        filter_action="native",            
        #filter_options=({"case":"sensitive"}),
        sort_action="native", #user can sort columns
        selected_rows=[],
        selected_columns=[],
        sort_mode="multi", #can be sorted across 'single' or 'multi' rows
        page_current=0, #page number user is on
        page_size=10, #number of rows visible per page
        style_cell={'textAlign': 'left'},
        #stylize table w/ Grazioso's colors
        style_header={ 
            'backgroundColor':'rgb(119,0,0)',
            'color': 'white',
            'fontWeight' : 'bold'
        },
        style_data={
            'backgroundColor':'rgb(237,237,237)',
            'color': 'black'
        }       

    ),
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Br(),
    html.Hr(),
    
    #Add footer/unique identifier and stylize
    html.Div(children='Created By: Sarah C Jodrey',
            style={                
                'color': 'black',
                'textAlign': 'center',
                'fontWeight': 'bold',
                'fontSize':15
            })
])



#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])


def update_dashboard(filter_type):

        #Logic filter for Water Rescue
        if filter_type == 'WR':
            df = pd.DataFrame.from_records(shelter.read({
                '$and':[{'breed':{'$in':['Chesa Bay Retr Mix',
                                         'Newfoundland Mix',
                                         'Labrador Retriever Mix']}},
                        {'sex_upon_outcome':'Intact Female'},
                        {'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}}]
            }))
            
        #Logic filter for Mountain/Wilderness Rescue    
        elif filter_type == 'MWR':            
            df = pd.DataFrame.from_records(shelter.read({
                '$and':[{'breed':{'$in':['German Shepherd',
                                        'Alaskan Malamute',
                                        'Old English Sheepdog',
                                        'Rottweiler',
                                        'Siberian Husky']}},
                        {'sex_upon_outcome':'Intact Male'},
                        {'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}}]
            }))
            
        #Logic filter for Disaster Rescue / Individual Tracking   
        elif filter_type == 'DRIT':             
            df = pd.DataFrame.from_records(shelter.read({
                '$and':[{'breed':{'$in':['German Shepherd',
                                        'Golden Retriever Mix',
                                        'Doberman Pinsch',
                                        'Rottweiler',
                                        'Bloodhound']}},
                        {'sex_upon_outcome':'Intact Male'},
                        {'age_upon_outcome_in_weeks':{'$gte':20,'$lte':300}}]
            }))
            
        #Return data table to initial state
        elif filter_type == 'RESET':
            df = pd.DataFrame.from_records(shelter.read({}))
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')        
        
        return (data,columns)



@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):    
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    
    return [
    #Creates pie chart based on selected data
       dcc.Graph(            
            figure = px.pie(data_frame= dff,
                            values= values,
                            names= names,
                            title= "Available for Rescue",
                            color_discrete_sequence = px.colors.sequential.RdBu,
                            width=500,height=500)
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return[
        dl.Map(style={'width':'750px', 'height':'400px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id='base-layer-id'),
            #Marker with tool tip and pop-up
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip([
                    html.H2("Animal Location"),
                    html.P(["Latitude:", dff.iloc[0,13]," Longitude:",dff.iloc[0,14]])
                           ]),
                dl.Popup([
                    html.H1("Animal Quick Facts"),
                    html.P(["Name: ",dff.iloc[0,9]]),
                    html.P(["Type: ", dff.iloc[0,3]]),
                    html.P(["Breed: ", dff.iloc[0,4]]),
                    html.P(["Sex: ", dff.iloc[0,12]]),
                    html.P(["Color: ", dff.iloc[0,5]]),
                    html.P(["Age: ", dff.iloc[0,1]])                   
                ])
            ])
        ])
    ]


app